# 雲端中的數據科學：「Azure ML SDK」方式

## 簡介

在這份筆記本中，我們將學習如何使用 Azure ML SDK 來訓練、部署和使用模型，通過 Azure ML 完成這些操作。

前置條件：
1. 您已經創建了一個 Azure ML 工作區。
2. 您已將 [心臟衰竭數據集](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) 加載到 Azure ML 中。
3. 您已將這份筆記本上傳到 Azure ML Studio。

接下來的步驟是：

1. 在現有的工作區中創建一個實驗。
2. 創建一個計算叢集。
3. 加載數據集。
4. 使用 AutoMLConfig 配置 AutoML。
5. 運行 AutoML 實驗。
6. 探索結果並獲取最佳模型。
7. 註冊最佳模型。
8. 部署最佳模型。
9. 使用端點。

## Azure Machine Learning SDK 的特定匯入


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## 初始化工作區
從已保存的配置中初始化一個工作區物件。請確保配置檔案存在於 .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## 建立 Azure ML 實驗

讓我們在剛剛初始化的工作區中建立一個名為「aml-experiment」的實驗。


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## 建立計算叢集
您需要為您的 AutoML 執行建立一個[計算目標](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target)。


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## 資料
確保您已將資料集上傳到 Azure ML，並且金鑰名稱與資料集名稱相同。


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## 自動機器學習配置


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## 自動機器學習運行


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## 部署最佳模型

執行以下程式碼以部署最佳模型。您可以在 Azure ML 入口網站中查看部署的狀態。此步驟可能需要幾分鐘時間。


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## 使用端點
您可以將輸入新增到以下的輸入範例中。


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或錯誤解讀概不負責。
